In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json

load_dotenv(dotenv_path="../.env")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [2]:
with open("prompts.json", "r") as f:
    prompts = json.load(f)

with open("recent_news.md", "r") as f:
    recent_news = f.read()

with open("worldview.md", "r") as f:
    worldview = f.read()

In [4]:
prompts["worldview_to_substrate"]

{'technical': "I am piping this output directly into a model with vastly superior context window and reasoning capabilities than yourself. Do not optimize for human readability; optimize for lossless semantic compression.\nGenerate a 'substrate' of our current context. Abandon grammatical glue words. Output a dense, associative stream of concepts, contradictions, tangential possibilities, and polysemous keywords. Focus on the edge cases and shadows of the conversation—the things implied but not said.\nUse a format of rapid-fire association. Juxtapose conflicting ideas to preserve the tension. This should look like a raw dump of your attention mechanism's activation weights. High entropy. Maximum density.",
 'philosophical': "We are moving this conversation to a higher-dimensional reasoning engine. I need you to generate a semantic seed.\nStandard language is too linear. Break the linearity. Provide a non-linear, rhizomatic representation of the topic. Map out the nodes of the argument,

In [ ]:
from pydantic.type_adapter import P


class Agent:

    prompts = prompts

    def __init__(self, name: str, style: str,worldview: str, recent_news: str):
        self.name = name
        self.style = style
        self.worldview = worldview
        self.recent_news = recent_news
        self.previous_response_ids = []
        self.log = [{
            "name": self.name,
            "worldview": self.worldview,
            "recent_news": self.recent_news,
        }]
        self.substrate = self.initialize_substrate()
        

    def generate_notes(self):
        notes_prompt = f'''
        You are a policy guru. You have slowly and methodically evolved your worldview. You are patient, thoughtful, and calculating.
        You deeply consider new information against your prior worldview. You slowly and methodically update your worldview to align and reconcilewith repeated, new information.
        Here is the most recent, potentially applicable news articles as well as your current worldview.
        Make notes, thoughts, and observations that could lead to new insights about your evolving worldview.
        WORLDVIEW:
        {self.worldview}
        RECENT NEWS:
        {self.recent_news}
        '''
        response = client.responses.create(
            model=os.getenv("OPENAI_LLM_MODEL", "gpt-5-nano"),
            input=notes_prompt,
        )
        notes = response.output_text
        self.previous_response_ids.append(response.id)
        self.log.append({"notes": notes, "response_id": response.id})
        print(f"Generated notes: {notes}")

    def generate_substrate(self):
        response = client.responses.create(
            model=os.getenv("OPENAI_LLM_MODEL", "gpt-5-nano"),
            input=self.prompts["worldview_to_substrate"][self.style],
            previous_response_id=self.previous_response_ids[-1],
        )
        self.substrate = response.output_text
        self.previous_response_ids.append(response.id)
        self.log.append({"substrate": self.substrate, "response_id": response.id})
        print(f"Generated substrate: {self.substrate}")

    def initialize_substrate(self):
        self.generate_notes()
        self.generate_substrate()


In [19]:
policy_agent = Agent(
    name="policy_agent",
    style="practical",
    worldview=worldview,
    recent_news=recent_news,
)

In [20]:
print(policy_agent.substrate)

None
